# Orca2 4 Prompt Generation

## Imports

In [1]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import LlamaCpp

## Setup

In [2]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

orca_path = "/Users/niccolominetti/Desktop/GTL-machine/llama.cpp/orca-2-7b.Q4_K_M.gguf"
#code_llama_path = "/Users/niccolominetti/Desktop/GTL-machine/llama.cpp/codellama-7b-instruct.Q2_K.gguf"
n_gpu_layers = 1  # Metal set to 1 is enough.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.

In [12]:
# Make sure the model path is correct for your system!
orca = LlamaCpp(
    model_path=orca_path,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx= 1024, # ctx_size=1024,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
    seed=81637,
    #repeat_penalty=1,
)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /Users/niccolominetti/Desktop/GTL-machine/llama.cpp/orca-2-7b.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 32003,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q4_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q4_K     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    7:            blk.0.ffn_down.

## Test

In [44]:
def prompt_gen_builder(task):
    prompt = f"""[INST] <<SYS>> You are an individual dedicated to expanding our understanding of the world and pushing the boundaries of human knowledge.
For the task specified below, outline a set of instructions to successfully achieve it.
Based on the task given, the goal is to correctly write a prompt form to generate an answer from an digital agent. 
The structure should consist of an example section with input : output pairs to show the correct behaviour.<</SYS>>

Task: Given a 'Description' of a python function, write the code.
Examples
Input: Description: A function which takes two numbers and multiplies them.
Output: def multiply(a,b):
    return a*b
Input: Description: A Python function to calculate the factorial of a number.
Output: def factorial(n):
    return 1 if n == 0 else n * factorial(n-1)

Task: {task}
Examples
Input:"""
    return prompt



### Given a macro task, write examples of input of how to achieve such task

In [45]:
task_example = "Write a list of steps to achieve a given task."
prompt = prompt_gen_builder(task_example)
print(prompt)

[INST] <<SYS>> You are an individual dedicated to expanding our understanding of the world and pushing the boundaries of human knowledge.
For the task specified below, outline a set of instructions to successfully achieve it.
Based on the task given, the goal is to correctly write a prompt form to generate an answer from an digital agent. 
The structure should consist of an example section with input : output pairs to show the correct behaviour.<</SYS>>

Task: Given a 'Description' of a python function, write the code.
Examples
Input: Description: A function which takes two numbers and multiplies them.
Output: def multiply(a,b):
    return a*b
Input: Description: A Python function to calculate the factorial of a number.
Output: def factorial(n):
    return 1 if n == 0 else n * factorial(n-1)

Task: Write a list of steps to achieve a given task.
Examples
Input:


In [46]:
output = orca(prompt, stop=["\n"])

Llama.generate: prefix-match hit


 Task: Create a python program that calculates the area of a circle using the formula A = πr^2, where r is the radius of the circle.


llama_print_timings:        load time =    2633.37 ms
llama_print_timings:      sample time =      22.32 ms /    35 runs   (    0.64 ms per token,  1568.17 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    3888.66 ms /    35 runs   (  111.10 ms per token,     9.00 tokens per second)
llama_print_timings:       total time =    4224.83 ms


In [47]:
output

' Task: Create a python program that calculates the area of a circle using the formula A = πr^2, where r is the radius of the circle.'

### Given a macro task, and an example input, write an example output

In [48]:
prompt_with_input = prompt + output + "\nOutput: "
print(prompt_with_input)

[INST] <<SYS>> You are an individual dedicated to expanding our understanding of the world and pushing the boundaries of human knowledge.
For the task specified below, outline a set of instructions to successfully achieve it.
Based on the task given, the goal is to correctly write a prompt form to generate an answer from an digital agent. 
The structure should consist of an example section with input : output pairs to show the correct behaviour.<</SYS>>

Task: Given a 'Description' of a python function, write the code.
Examples
Input: Description: A function which takes two numbers and multiplies them.
Output: def multiply(a,b):
    return a*b
Input: Description: A Python function to calculate the factorial of a number.
Output: def factorial(n):
    return 1 if n == 0 else n * factorial(n-1)

Task: Write a list of steps to achieve a given task.
Examples
Input: Task: Create a python program that calculates the area of a circle using the formula A = πr^2, where r is the radius of the cir

In [49]:
output_out = orca(prompt_with_input, stop=[])

Llama.generate: prefix-match hit


1. Import the math library in Python using the command "import math".
2. Declare and assign a variable for the radius of the circle, say 'radius', and assign it a value, for example, 'radius = 5'.
3. Call the formula A = πr^2 by assigning the value of the radius to the variable r and passing it as an argument to the formula.
4. Store the result in a variable, say 'area', and print it using the print() function.


llama_print_timings:        load time =    2633.37 ms
llama_print_timings:      sample time =      74.45 ms /   108 runs   (    0.69 ms per token,  1450.70 tokens per second)
llama_print_timings: prompt eval time =    1178.84 ms /     4 tokens (  294.71 ms per token,     3.39 tokens per second)
llama_print_timings:        eval time =    8848.66 ms /   107 runs   (   82.70 ms per token,    12.09 tokens per second)
llama_print_timings:       total time =   11467.30 ms


### Example pair

In [88]:
# Example input
print(output)

 Task: Create a python program that calculates the area of a circle using the formula A = πr^2, where r is the radius of the circle.


In [89]:
# Example input
print(output_out)

1. Import the math library in Python using the command "import math".
2. Declare and assign a variable for the radius of the circle, say 'radius', and assign it a value, for example, 'radius = 5'.
3. Call the formula A = πr^2 by assigning the value of the radius to the variable r and passing it as an argument to the formula.
4. Store the result in a variable, say 'area', and print it using the print() function.


### Let's put this all in a function

In [90]:
def gen_task_example(model="", task=""):
    # Check if input string is empty
    if task == "": return None
    if model == "": return None

    # Given a task, we want to build a prompt which shows the LLM how prompts are built
    prompt = prompt_gen_builder(task_example)
    # Call the model. Given this prompt, we want to generate an input example
    input_example_response = model(prompt, stop=["\n"])
    # Add the response to the prompt and add the Output string to ellicit the output of the example.
    prompt_with_input = prompt + input_example_response + "\nOutput: "
    # Call the model. Given this prompt, we want to generate an output example
    output_example_response = model(prompt_with_input, stop=[])

    return input_example_response, output_example_response

In [91]:
# Test the function
task  = "Given a description of a python function, write the function in a concise way."
examples = []
for n in range(5):
    example_input, example_output = gen_task_example(model=orca, task=task)
    examples.append([example_input, example_output])

Llama.generate: prefix-match hit


 Description: A Python function to check if two lists are equal.


llama_print_timings:        load time =    2633.37 ms
llama_print_timings:      sample time =      10.84 ms /    14 runs   (    0.77 ms per token,  1291.27 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    2773.29 ms /    14 runs   (  198.09 ms per token,     5.05 tokens per second)
llama_print_timings:       total time =    2913.88 ms
Llama.generate: prefix-match hit



def are_equal(list1, list2):
    return len(list1) == len(list2) and all(i in list1 for i in list2)

Task: Given a description of a python function, write the function in a concise way.
Examples
Input: Description: A Python function that returns True if any of the numbers in the list are multiples of 3 or 5.
Output: def are_multiples(numbers):
    return any(num % 3 == 0 or num % 5 == 0 for num in numbers) Description


llama_print_timings:        load time =    2633.37 ms
llama_print_timings:      sample time =      91.36 ms /   129 runs   (    0.71 ms per token,  1411.93 tokens per second)
llama_print_timings: prompt eval time =     316.79 ms /     4 tokens (   79.20 ms per token,    12.63 tokens per second)
llama_print_timings:        eval time =   10650.80 ms /   128 runs   (   83.21 ms per token,    12.02 tokens per second)
llama_print_timings:       total time =   12791.07 ms
Llama.generate: prefix-match hit


...


llama_print_timings:        load time =    2633.37 ms
llama_print_timings:      sample time =       8.84 ms /    15 runs   (    0.59 ms per token,  1697.02 tokens per second)
llama_print_timings: prompt eval time =     281.44 ms /     4 tokens (   70.36 ms per token,    14.21 tokens per second)
llama_print_timings:        eval time =    1167.05 ms /    14 runs   (   83.36 ms per token,    12.00 tokens per second)
llama_print_timings:       total time =    1591.93 ms
Llama.generate: prefix-match hit


 A function which takes two numbers and multiplies them.


llama_print_timings:        load time =    2633.37 ms
llama_print_timings:      sample time =      10.89 ms /    14 runs   (    0.78 ms per token,  1285.35 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    1104.19 ms /    14 runs   (   78.87 ms per token,    12.68 tokens per second)
llama_print_timings:       total time =    1239.09 ms
Llama.generate: prefix-match hit



def multiply(a,b):
    return a*b


llama_print_timings:        load time =    2633.37 ms
llama_print_timings:      sample time =      10.44 ms /    15 runs   (    0.70 ms per token,  1436.92 tokens per second)
llama_print_timings: prompt eval time =     281.89 ms /     4 tokens (   70.47 ms per token,    14.19 tokens per second)
llama_print_timings:        eval time =    1093.60 ms /    14 runs   (   78.11 ms per token,    12.80 tokens per second)
llama_print_timings:       total time =    1513.99 ms


In [98]:
def task_and_examples_to_prompt(task="", examples=""):
    # Check if inputs are valid
    if task == "": return None
    if examples == "": return None

    prompt = f"""[INST] <<SYS>> You are an individual dedicated to expanding our understanding of the world and pushing the boundaries of human knowledge.
For the task specified below, outline a set of instructions to successfully achieve it.
Based on the task given, the goal is to correctly write a prompt form to generate an answer from an digital agent. 
The structure should consist of an example section with input : output pairs to show the correct behaviour.<</SYS>>

Task: {task}
Examples
"""
    for example in examples:
        prompt = prompt + f"Input: {example[0]}\nOutput:{example[1]}\n"

    prompt += "Input: "
    return prompt

In [99]:
auto_prompt_response = task_and_examples_to_prompt(task, examples)

In [100]:
print(auto_prompt_response)

[INST] <<SYS>> You are an individual dedicated to expanding our understanding of the world and pushing the boundaries of human knowledge.
For the task specified below, outline a set of instructions to successfully achieve it.
Based on the task given, the goal is to correctly write a prompt form to generate an answer from an digital agent. 
The structure should consist of an example section with input : output pairs to show the correct behaviour.<</SYS>>

Task: Given a description of a python function, write the function in a concise way.
Examples
Input:  Description: A Python function to check if two lists are equal.
Output:
def are_equal(list1, list2):
    return len(list1) == len(list2) and all(i in list1 for i in list2)

Task: Given a description of a python function, write the function in a concise way.
Examples
Input: Description: A Python function that returns True if any of the numbers in the list are multiples of 3 or 5.
Output: def are_multiples(numbers):
    return any(num % 